In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./data/proposal.pdf') #  data 폴더 안에 들어있습니다.
pdf_docs = pdf_loader.load()

print(len(pdf_docs)) # number of documents
print()
print(type(pdf_docs)) # list of documents
print()
print(pdf_docs[0]) # document

2

<class 'list'>

page_content='file.md 2025-09-09
1 / 2
프로젝트  개요 : K-Culture 팬  창작자  보상  및  활동  증명 
플랫폼 
1. 프로젝트  비전  (The 'Why')
K-POP 의  성장을  이끄는  숨은  주역인  팬  창작자들의  열정과  기여가  정당한  가치로  인정받는  생태계 를  만듭니 
다 . 우리는  저작권  문제를  지혜롭게  해결하면서 , 이들의  창작  활동이  투명한  보상 과  영구적인  명예 로  이어지는 
새로운  모델을  제시합니다 .
2. 문제  정의와  핵심  해결책 
문제 : 댄스  커버 , 팬아트  등  2 차  창작물은  원저작권  때문에  직접적인  ' 판매 ' 나  ' 수익화 ' 가  불가능합니다 . 이 
로  인해  창작자들의  기여는  정당한  보상으로  이어지지  못하고  휘발되기  쉽습니다 . 또한  K-POP 안무  또한 
저작권으로  보호  되지만 , 음악에  비해서  저작권  인식이  높지  않다 .
핵심  해결책 : ' 판매 ' 가  아닌  ' 시상 (Reward)' 모델 
우리는  저작권이  있는  2 차  창작물을  거래하는  NFT 마켓플레이스를  만들지  않습니다 .
대신 , ' 댄스  경연대회 ' 모델을  적용합니다 . 참가자들이  기존  곡으로  실력을  뽐내고  **' 상금 '** 을  받 
는  것처럼 , 팬  창작자들은  자신의  활동과  인기에  따라  ' 보상  풀 (Reward Pool)' 에  모인  재원 을  받게 
됩니다 .
이  모델을  통해  우리는  저작권  침해  없이 , 창작자의  **' 활동 ' 과  ' 기여 '** 에  대해  보상할  수  있습니다 .
3. ' 보상  풀 ' 의  재원은  어디서  오는가 ? ( 상금의  출처 )
이  상금은  **' 팬덤  활성화 '** 를  목표로  하는  다양한  주체로부터  나옵니다 .
1. 아티스트 / 소속사 : 신곡  홍보 , 챌린지  이벤트  등  마

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200, # 문맥을 살리기 위해서 겹치는 부분 크기
    length_function = len, # 글자 수를 기준 -> 이걸 왜씀? 청크 사이즈 1000이 글자수 1000이 되도록 한다.
    separators = ["\n\n", "\n", " ", ""], # 여러 기준으로 청크 나누기가 가능, 앞에 나오는 것 우선 -> 이게 RecursiveCharacterTextSplitter의 핵심 기능
)

texts = text_splitter.split_documents(pdf_docs) # 위에서 읽은 pdf
print(f"생성된 텍스트 청크 수 : {len(texts)}")
print(f"각 청크의 길이 : {list(len(text.page_content) for text in texts)}")

생성된 텍스트 청크 수 : 3
각 청크의 길이 : [927, 902, 774]


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model = "text-embedding-3-large")

embeddings_model

In [ ]:
from langchain_chroma import Chroma

# 처음 데이터 생성
vector_store = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    collection_name='chroma-req',
    persist_directory='./chroma-req'
)

# 데이터 로딩
# vector_store = Chroma(
#     collection_name='chroma-req',
#     embedding_function=embeddings_model,
#     persist_directory='./chroma-req'
# )

retriever = vector_store.as_retriever(search_kwargs={'k':3})